In [1]:
!pip install pandera

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.5/148.5 kB 3.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 8.4 MB/s eta 0:00:00
  Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)


In [6]:
import pandas as pd
import pandera as pa
from pandera import Column, Check

Imagine we have a DataFrame with a set of conditions:

In [4]:
fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": [2, 1, 3, 4]
    }
)

available_fruits = ["apple", "banana", "orange"] # Possible values for name column
nearby_stores = ["Aldi", "Walmart"] # Possible values for store column

We can now check if our dataframe satisfies the conditions with `pandera`

In [7]:
schema = pa.DataFrameSchema( #We create a DataFrameSchema to validate our DataFrame
    {
        "name": Column(str, Check.isin(available_fruits)), #We check the Column name, etc
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(int, Check.less_than(4)) 
        
    }
)

schema.validate(fruits) # This fails because there is a value in the price column equal to 4!

SchemaError: <Schema Column(name=price, type=DataType(int64))> failed element-wise validator 0:
<Check less_than: less_than(4)>
failure cases:
   index  failure_case
0      3             4

In [9]:
schema2 = pa.DataFrameSchema( #We create a DataFrameSchema to validate our DataFrame
    {
        "name": Column(str, Check.isin(available_fruits)), #We check the Column name, etc
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(int, Check.less_than(5)) 
        
    }
)

schema2.validate(fruits) # If we may set it to 5 this works!

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4


We can also check multiple values at the same time:

In [10]:
schema3 = pa.DataFrameSchema( #We create a DataFrameSchema to validate our DataFrame
    {
        "name": Column(str, Check.isin(available_fruits)), #We check the Column name, etc
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(int, [
            Check.less_than(5),
            Check.greater_than(4)
            ]
        )
    }
)

schema3.validate(fruits) # If we may set it to 5 this works!

SchemaError: <Schema Column(name=price, type=DataType(int64))> failed element-wise validator 1:
<Check greater_than: greater_than(4)>
failure cases:
   index  failure_case
0      0             2
1      1             1
2      2             3
3      3             4

We can create a custom check using `lambda` functions:

In [12]:
schema4 = pa.DataFrameSchema( #We create a DataFrameSchema to validate our DataFrame
    {
        "name": Column(str, Check.isin(available_fruits)), #We check the Column name, etc
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(int, [
            Check.less_than(5),
            Check(lambda price: sum(price) < 20) # Check sum of all prices is lower than 20
            ]
        )
    }
)

schema4.validate(fruits) # If we may set it to 5 this works!

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4


## References

- [GitHub](https://github.com/unionai-oss/pandera)
- [Documentation](https://pandera.readthedocs.io/en/stable/index.html)
